In [1]:
import pandas as pd
import numpy as np
 
import tensorflow as tf
 
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("diamonds.csv")

In [3]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

34521 train examples
8631 validation examples
10788 test examples


In [4]:
#Membuat kolom target
# 0 = low; 1 = high
df['target'] = np.where(df['price'] <= 1000, 0, 1)
# Drop un-used columns.
df = df.drop(columns=['price'])

In [5]:
# Cara untuk membuat dataset tf.data dari pandas dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = df.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds
batch_size = 10 #bath ukuran kecil untuk demonstrasi
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [6]:
example_batch = next(iter(train_ds))[0]
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

In [7]:
#numeric column
carat = feature_column.numeric_column('carat')
demo(carat)

[[0.7 ]
 [0.7 ]
 [1.2 ]
 [1.19]
 [0.71]
 [0.71]
 [0.37]
 [0.36]
 [2.01]
 [1.  ]]


In [8]:
#bucketized column
carat = feature_column.numeric_column('carat')
carat_buckets = feature_column.bucketized_column(carat, boundaries=[1, 2])
demo(carat_buckets)

[[1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]


In [9]:
#categorical
color_type = feature_column.categorical_column_with_vocabulary_list(
      'color', ['E', 'I','J','D','H', 'G','F'])
 
color_type_one_hot = feature_column.indicator_column(color_type)
demo(color_type_one_hot)

[[0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]]


In [10]:
#embedding
clarity = feature_column.categorical_column_with_vocabulary_list(
      'clarity', df.clarity.unique())
clarity_embedding = feature_column.embedding_column(clarity, dimension=6)
demo(clarity_embedding)

[[ 0.06596094 -0.25000304 -0.09386437 -0.05818721 -0.39615995 -0.5487775 ]
 [-0.49688536  0.23828071 -0.10483676  0.04456249  0.43668735  0.56724215]
 [-0.49688536  0.23828071 -0.10483676  0.04456249  0.43668735  0.56724215]
 [-0.49688536  0.23828071 -0.10483676  0.04456249  0.43668735  0.56724215]
 [ 0.06596094 -0.25000304 -0.09386437 -0.05818721 -0.39615995 -0.5487775 ]
 [ 0.22822225  0.2023663  -0.02079243 -0.31212884 -0.24685124 -0.03681855]
 [-0.49688536  0.23828071 -0.10483676  0.04456249  0.43668735  0.56724215]
 [ 0.06596094 -0.25000304 -0.09386437 -0.05818721 -0.39615995 -0.5487775 ]
 [ 0.22822225  0.2023663  -0.02079243 -0.31212884 -0.24685124 -0.03681855]
 [-0.49688536  0.23828071 -0.10483676  0.04456249  0.43668735  0.56724215]]


In [11]:
#hashed feature
clarity_hashed = feature_column.categorical_column_with_hash_bucket(
      'clarity', hash_bucket_size=5)
demo(feature_column.indicator_column(clarity_hashed))

[[1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]]


In [12]:
#cross feature
#data yang di cross harus berupa string, categorical, atau bucketized
crossed_feature = feature_column.crossed_column([carat_buckets, color_type],
                                                hash_bucket_size=10)
demo(feature_column.indicator_column(crossed_feature))

[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]


In [13]:
#Pilih feature column mana yang akan digunakan
feature_columns = []
# numeric column
for header in ['carat', 'depth', 'x', 'y', 'z']:
  feature_columns.append(feature_column.numeric_column(header))
 
#membuat feature layer
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [14]:
#create, compile, and train the model
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])
 
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
 
model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

Epoch 1/10
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
1685/1686 [============================>.] - ETA: 0s - loss: 0.2043 - accuracy: 0.9115WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'Unnamed: 0': <tf.Tensor 'IteratorGetNext:0' shape=(None,) dtype=int64>, 'carat': <tf.Tensor 'IteratorGetNext:1' shape=(None,) dtype=float64>, 'cut': <tf.Tensor 'IteratorGetNext:4' shape=(None,) dtype=string>, 'color': <tf.Tensor 'IteratorGetNext:3' shape=(None,) dtype=string>, 'clarity': <tf.Tensor 'IteratorGetNext:2' shape=(None,) dtype=string>, 'depth': <tf.Tensor 'IteratorGetNext:5' shape=(None,) dtype=float64>, 'table': <tf.Tensor 'IteratorGetNext:6' shape=(None,) dtype=float64>, 'x': <tf.Tensor 'IteratorGetNext:7' shape=(None,) dtype=float64>, 'y': <tf.Tensor 'IteratorGetNext:8' shape=(None,) dtype=float64>, 'z': <tf.Tensor 'IteratorGetNext:9' shape=(